In [5]:
import sys

sys.path.append("../src")

import time
from api_client import create_openai_client, create_google_service, get_openai_response_and_total_tokens, search_google
from helpers import parse_html
from dotenv import dotenv_values
import requests

config = dotenv_values("../.env") 

num_results = 5

open_ai_key = config['OPEN_AI_API_KEY']
google_api_key = config['GOOGLE_API_KEY']
search_engine_id = config['SEARCH_ENGINE_ID']

# Initialize clients
openai_client = create_openai_client(open_ai_key)
google_service, search_id = create_google_service(google_api_key, search_engine_id)

original_query = "life insurance products offered by state farm"
original_query = "life insurance products offered by southern farm bureau life insurance company"

# Step 1: Rephrase query using OpenAI
messages = [
    {"role": "system", "content": "Rephrase the user's query for search engine optimization."},
    {"role": "user", "content": original_query},
]
rephrased_query, total_tokens_from_rephrasing = get_openai_response_and_total_tokens(openai_client, messages,  model="gpt-4o")

print(rephrased_query)
print(total_tokens_from_rephrasing)

What life insurance plans are available from Southern Farm Bureau Life Insurance Company?
46


In [6]:
# Step 2: Search Google
search_results = search_google(google_service, search_id, rephrased_query)

# Step 3: Fetch content and parse HTML

refined_sources = []

for i in [0, 1]:
    
    time.sleep(2)
    
    url = search_results[i]['link']
    response = requests.get(url)
    raw_text = parse_html(response.content)

    messages = [
    {"role": "system", "content": f"Refine the following text and extract only relevant information related to the following query: {rephrased_query}"},
    {"role": "user", "content": raw_text},
]

    refined_text, total_tokens_from_refining = get_openai_response_and_total_tokens(openai_client, messages, model="gpt-4o-mini")
    
    print(refined_text)
    print(total_tokens_from_refining)

    refined_sources.append({"url" : url, "refined_text" : refined_text})

Southern Farm Bureau Life Insurance Company offers a range of life insurance plans, including:

- Term Life Insurance
- Whole Life Insurance
- Universal Life Insurance

These products are designed to provide competitive options and superior customer service to policyholders and agents. For specific details or to find an agent, individuals can visit their website or contact the company directly.
303
I'm unable to provide information about life insurance plans from Southern Farm Bureau Life Insurance Company as the content is restricted. Please refer to the company's official website or contact them directly for detailed information on available life insurance plans.
93


In [7]:
messages = [
    {"role": "system", "content": "Write an academic-style response to the query, citing the refined sources."},
    {"role": "user", "content": f"Original Query: {rephrased_query}\nRefined Sources: {refined_sources}"},
]

final_response, total_tokens_from_final_response = get_openai_response_and_total_tokens(openai_client, messages,  model="gpt-4o")

In [8]:
print(final_response)

Southern Farm Bureau Life Insurance Company offers a range of life insurance products designed to meet diverse financial needs and preferences. Their offerings primarily include Term Life Insurance, Whole Life Insurance, and Universal Life Insurance (Southern Farm Bureau Life Insurance Company, n.d.).

1. **Term Life Insurance**: This option provides coverage for a specific period, making it an attractive choice for individuals seeking protection during peak responsibility years such as raising children or paying off a mortgage. Term life policies are typically more affordable compared to permanent life insurance options and serve as a straightforward method to ensure financial security for one's beneficiaries in the event of the policyholder's untimely death.

2. **Whole Life Insurance**: This policy is a form of permanent life insurance that offers lifetime coverage, with the added component of accumulating cash value over time. Policyholders can benefit from consistent premium payme